# Using Xtract to index research artifacts stored on Jetstream Cloud


### This Xtract-Jetstream demo illustrates how to crawl, extract metadata from, and ingest metadata for any Globus Endpoint.

#### We begin by importing important libraries. Of note, we use the `mdf_toolbox` library as a wrapper for Globus Auth. 

In [549]:
import time
import json
import requests
import mdf_toolbox

## Step 0: Configuration

#### Here we provide configuration details for our metadata extraction job, including specifications for both Globus and funcX.

In [550]:
# JETSTREAM: Globus endpoint and directory path AND funcX endpoint where the data reside
source_ep_path_1 = "/home/tskluzac/tyler_research_files" 
local_mdata_dir = "/home/tskluzac/mdata"
source_ep_id = "f9959bd2-e98f-11eb-884c-aba19178789c"  # Globus ID for Jetstream instance 1
funcx_ep_id = "e1398319-0d0f-4188-909b-a978f6fc5621"  # funcX ID for Jetstream instance 1



# PETREL: Globus endpoint and file path at which we want to archive metadata documents
mdata_ep_id = "4f99675c-ac1f-11ea-bee8-0e716405a293"  # Xtract Petrel EP: 4f...93
remote_mdata_dir = "/home/my_metadata"

# CRAWLER URL:
eb_crawl_url = "http://xtractcrawler5-env.eba-akbhvznm.us-east-1.elasticbeanstalk.com"
eb_extract_url = "http://127.0.0.1:5000"

# GROUPER: Grouping strategy we want to use for grouping. "file_is_group" means each file is a distinct data entity. 
grouper = "file_is_group"

## Step 1: Login 

Here we use `mdf_toolbox` to request tokens from Globus Auth. When fresh tokens are needed, users will authenticate with their Globus ID by following the directions in the STDOUT. Notable auth scopes are as follows: 

* **openid**: provides username for identity.
* **search**: interact with Globus Search
* **petrel**: read or write data on Petrel. Not needed if no data going to Petrel.
* **transfer**: needed to crawl the Globus endpoint and transfer metadata to its final location.
* **funcx_scope**: needed to orchestrate the metadata exraction at the given funcX endpoint.

The following code block initializes all of the tokens.

In [546]:
print("Authenticating...")
funcx_scope = "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all"
auths = mdf_toolbox.login(
    services=[
        "openid",
        "data_mdf",
        "search",
        "petrel",
        "transfer",
        "dlhub",
        funcx_scope,
    ],
    app_name="Foundry",
    make_clients=True,
    no_browser=False,
    no_local_server=False,
)
print("Authentication successful!")
print(auths)

Authenticating...
Authentication successful!
{'search': <globus_sdk.search.client.SearchClient object at 0x7fc728143240>, 'data_mdf': <globus_sdk.authorizers.refresh_token.RefreshTokenAuthorizer object at 0x7fc70815b400>, 'petrel': <globus_sdk.authorizers.refresh_token.RefreshTokenAuthorizer object at 0x7fc70815b2b0>, 'dlhub': <globus_sdk.authorizers.refresh_token.RefreshTokenAuthorizer object at 0x7fc70815b5c0>, 'transfer': <globus_sdk.transfer.client.TransferClient object at 0x7fc70815bba8>, 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all': <globus_sdk.authorizers.refresh_token.RefreshTokenAuthorizer object at 0x7fc70815b710>, 'openid': <globus_sdk.authorizers.refresh_token.RefreshTokenAuthorizer object at 0x7fc70815b780>}


## Step 2: Crawl
Crawling, behind the scenes, will scan a Globus directory breadth-first (using globus_ls), first extracting physical metadata such as path, size, and extension. Next, since the *grouper* we selected is 'file_is_group', the crawler will simply create `n` single-file groups. 

The crawl is **non-blocking**, and the crawl_id here will be used to execute and monitor downstream extraction processes. 

In [563]:
crawl_url = f'{eb_url}/crawl'
print(f"Crawl URL is : {crawl_url}")

first_ep_dict = {
    'repo_type': 'GLOBUS',
    'eid': source_ep_id,
    'dir_paths': [source_ep_path_1], # Can add more than one path to this list. 
    'grouper': grouper
}

crawl_tokens = {'Transfer': auths['transfer'].authorizer.access_token, 
          'Authorization': f"Bearer {auths['transfer'].authorizer.access_token}", 
          'FuncX': auths[fx_scope].access_token}


crawl_req = requests.post(crawl_url, json={'endpoints': [first_ep_dict], 'tokens': crawl_tokens})
crawl_id = json.loads(crawl_req.content)['crawl_id']
print(f"Crawl ID: {crawl_id}")

Crawl URL is : http://xtractcrawler5-env.eba-akbhvznm.us-east-1.elasticbeanstalk.com/crawl
Crawl ID: 9aea1531-ea1a-4751-bc67-c0b364fc31b2


We can get crawl status, seeing how many groups have been identified in the crawl. 

Note that measuring the total files yet to crawl is impossible, as the BFS may not have discovered all files yet, and Globus does not yet have a file counting feature for all directories and subdirectories. I.e., we know when we're done, but we do not know until we get there. 

**Warning:** it currently takes up to 30 seconds for a crawl to start. *Why?* Container warming time. 

In [564]:
while True: 
    crawl_status = requests.get(f'{eb_url}/get_crawl_status', json={'crawl_id': crawl_id})
    print(crawl_status)
    crawl_content = json.loads(crawl_status.content)
    print(f"Crawl Status: {crawl_content}")
    
    # Break the loop if we collected 
    if crawl_content['crawl_status'] == 'complete':
        break
    time.sleep(2)

<Response [200]>
Crawl Status: {'bytes_crawled': 0, 'crawl_id': '9aea1531-ea1a-4751-bc67-c0b364fc31b2', 'crawl_status': 'crawling', 'files_crawled': 0, 'groups_crawled': 0}
<Response [200]>
Crawl Status: {'bytes_crawled': 20146644, 'crawl_id': '9aea1531-ea1a-4751-bc67-c0b364fc31b2', 'crawl_status': 'crawling', 'files_crawled': 50, 'groups_crawled': 50}
<Response [200]>
Crawl Status: {'bytes_crawled': 20146644, 'crawl_id': '9aea1531-ea1a-4751-bc67-c0b364fc31b2', 'crawl_status': 'crawling', 'files_crawled': 50, 'groups_crawled': 50}
<Response [200]>
Crawl Status: {'bytes_crawled': 20146644, 'crawl_id': '9aea1531-ea1a-4751-bc67-c0b364fc31b2', 'crawl_status': 'complete', 'files_crawled': 50, 'groups_crawled': 50}


## Step 3a. You can directly flush the crawl metadata via REST 

#### Why? Downloading crawl metadata is useful for many file organization tasks, such as: 
- I want a list of all files on my file system
- I want to know the total size (GB) of a folder
- I want to tally files by extension

#### Currently Foundry uses Xtract to create a list of all files in user-submitted folders. Check it out here: 
TODO: LINK TO FOUNDRY. 

**Caution**: if you flush the crawl metadata (3a), **you may not** extract metadata from them (3b). If you want to do both, you must launch two separate crawl jobs. 

In [71]:
while True:
    req = requests.get(f'{eb_url}/fetch_crawl_mdata', json={'crawl_id': crawl_id, 'n': 100})
    print(req.content)
    time.sleep(1)

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710512.1661508,"file_id":"9fbc12cb-15a5-4a3c-87de-b7769f654c49","metadata":{"physical":{"extension":"py","path_type":"globus","size":4325}},"path":"/home/tskluzac/ext_repos/xtract-hdf/xtract_hdf_main.py","size":4325},{"base_url":null,"crawl_timestamp":1635710512.6035132,"file_id":"93dd7fc2-bd7c-446f-93c1-5512958d7098","metadata":{"physical":{"extension":"py","path_type":"globus","size":3979}},"path":"/home/tskluzac/ext_repos/xtract-images/xtract_images_main.py","size":3979},{"base_url":null,"crawl_timestamp":1635710512.603453,"file_id":"5bd5c116-f9d4-4589-9970-483dd198f0cc","metadata":{"physical":{"extension":"img","path_type":"globus","size":770920448}},"path":"/home/tskluzac/ext_repos/xtract-images/xtract-images.img","size":770920448},{"base_url":null,"crawl_timestamp":1635710512.7

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710519.3185995,"file_id":"5baacd5b-a48b-473b-aff7-33abdc836430","metadata":{"physical":{"extension":"yml","path_type":"globus","size":260}},"path":"/home/tskluzac/ext_repos/xtract-matio/data-schemas/.travis.yml","size":260},{"base_url":null,"crawl_timestamp":1635710519.321685,"file_id":"ff6b4383-1bb0-4bbb-88ed-825659b3f5f1","metadata":{"physical":{"extension":null,"path_type":"globus","size":828}},"path":"/home/tskluzac/ext_repos/xtract-netcdf/.git/index","size":828},{"base_url":null,"crawl_timestamp":1635710519.3575697,"file_id":"96252882-9dce-4faa-8519-8a26a21d0fbe","metadata":{"physical":{"extension":"yml","path_type":"globus","size":272}},"path":"/home/tskluzac/ext_repos/xtract-matio/mdf-materialsio-adapters/.travis.yml","size":272},{"base_url":null,"crawl_timestamp":1635710519.3735552,"file_id":"13d2d5b0-d2f8-4926-9dfc-cd178810947d","metadata":{"physical":{"extension":null,"path_

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710519.7670228,"file_id":"bab9cfdd-ac8a-429e-a113-09d4f673a90f","metadata":{"physical":{"extension":null,"path_type":"globus","size":41}},"path":"/home/tskluzac/ext_repos/xtract-tabular/.git/ORIG_HEAD","size":41},{"base_url":null,"crawl_timestamp":1635710519.7671924,"file_id":"f67585b2-24e2-4456-bb67-89fd03259e17","metadata":{"physical":{"extension":null,"path_type":"globus","size":114}},"path":"/home/tskluzac/ext_repos/xtract-tabular/.git/packed-refs","size":114},{"base_url":null,"crawl_timestamp":1635710519.7882953,"file_id":"a1ea0267-93e6-4de2-8133-3ac3e06254a4","metadata":{"physical":{"extension":"py","path_type":"globus","size":365}},"path":"/home/tskluzac/ext_repos/xtract-tika/legacy/batch.py","size":365},{"base_url":null,"crawl_timestamp":1635710519.7830107,"file_id":"ac580922-b045-4cb1-8ea9-ca0b8295d020","metadata":{"physical":{"extension":"py","path_type":"globus","size":2455

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710525.350724,"file_id":"623bd64a-caa5-4acf-9d9f-2a5ad5b92a31","metadata":{"physical":{"extension":"png","path_type":"globus","size":129436}},"path":"/home/tskluzac/ext_repos/xtract-images/training_data/map_plots/2015-05 PS92_Report-page-1.png","size":129436},{"base_url":null,"crawl_timestamp":1635710525.3508763,"file_id":"d8c6b6ae-362e-4b4f-9465-8bb941d3f5b5","metadata":{"physical":{"extension":"png","path_type":"globus","size":141010}},"path":"/home/tskluzac/ext_repos/xtract-images/training_data/map_plots/2015-08 PS94_MAP.png","size":141010},{"base_url":null,"crawl_timestamp":1635710525.6898124,"file_id":"f09edea0-a7ad-4794-9e38-8011c2f176d7","metadata":{"physical":{"extension":"jpg","path_type":"globus","size":189416}},"path":"/home/tskluzac/ext_repos/xtract-images/training_data/maps/49XK19940212.jpg","size":189416},{"base_url":null,"crawl_timestamp":1635710525.689895,"file_id":"39

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710525.7265437,"file_id":"4a0294ce-46fd-4748-bb9c-a772bb9b590e","metadata":{"physical":{"extension":"jpg","path_type":"globus","size":142592}},"path":"/home/tskluzac/ext_repos/xtract-images/training_data/photographs/12137639-happy-family-.jpg","size":142592},{"base_url":null,"crawl_timestamp":1635710525.7269251,"file_id":"5b267448-14f0-44f5-ada7-f06b7e342b27","metadata":{"physical":{"extension":"jpg","path_type":"globus","size":62315}},"path":"/home/tskluzac/ext_repos/xtract-images/training_data/photographs/3-ocean.jpg","size":62315},{"base_url":null,"crawl_timestamp":1635710525.7271478,"file_id":"cd96dde0-3491-4e9d-b5e3-64673e319080","metadata":{"physical":{"extension":"jpg","path_type":"globus","size":162905}},"path":"/home/tskluzac/ext_repos/xtract-images/training_data/photographs/40896795-family-with-thumbs-up.jpg","size":162905},{"base_url":null,"crawl_timestamp":1635710525.72722

...

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710549.979814,"file_id":"e2e7dd21-e138-4232-aaba-c27ce3be0b9b","metadata":{"physical":{"extension":null,"path_type":"globus","size":557}},"path":"/home/tskluzac/ext_repos/xtract-jsonxml/.git/objects/a7/25d766bcb49269e9ab9f600e6bc1e82f51d174","size":557},{"base_url":null,"crawl_timestamp":1635710542.1674519,"file_id":"e322bc3c-6016-4aa6-96a4-cf8a78726ccb","metadata":{"physical":{"extension":null,"path_type":"globus","size":563}},"path":"/home/tskluzac/ext_repos/xtract-keyword/.git/objects/d3/b8b2c7efd385684871c00077595d74f6b5e774","size":563},{"base_url":null,"crawl_timestamp":1635710541.188459,"file_id":"61a3a78b-cd5e-4af0-a9cc-4e2145f972ea","metadata":{"physical":{"extension":null,"path_type":"globus","size":388}},"path":"/home/tskluzac/ext_repos/xtract-keyword/.git/objects/a7/544ff0c5827c16e0ba25600b8ebc2e527b01d3","size":388},{"base_url":null,"crawl_timestamp":1635710542.0657291,"f

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710541.6210642,"file_id":"c0a5c1e3-103a-4604-8445-afb0d5a0c27b","metadata":{"physical":{"extension":null,"path_type":"globus","size":1308}},"path":"/home/tskluzac/ext_repos/xtract-keyword/.git/objects/b4/7e63c56a4fe149aceb697960b1ea5314e29925","size":1308},{"base_url":null,"crawl_timestamp":1635710536.2433264,"file_id":"c74f159a-d2fb-44cf-9461-cd8f7c232491","metadata":{"physical":{"extension":null,"path_type":"globus","size":580}},"path":"/home/tskluzac/ext_repos/xtract-images/.git/objects/69/83265ed7890c4e3ef0a18f69b026a57c24d5e5","size":580},{"base_url":null,"crawl_timestamp":1635710540.331458,"file_id":"173c24ba-4f78-4d03-bddf-d635c818205d","metadata":{"physical":{"extension":null,"path_type":"globus","size":315}},"path":"/home/tskluzac/ext_repos/xtract-keyword/.git/objects/77/1d9f6c5d8f0fac37ae9e9d3085509e9cfcd6f9","size":315},{"base_url":null,"crawl_timestamp":1635710542.0822837,

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[{"base_url":null,"crawl_timestamp":1635710556.8188522,"file_id":"b642b727-695e-47c3-a080-5f20c78d1774","metadata":{"physical":{"extension":null,"path_type":"globus","size":174}},"path":"/home/tskluzac/ext_repos/xtract-tika/.git/objects/ed/a94000fd179ef4f76bef796a3663e185575abf","size":174},{"base_url":null,"crawl_timestamp":1635710556.19483,"file_id":"93159657-d23f-4439-af8a-70524c19b7ae","metadata":{"physical":{"extension":null,"path_type":"globus","size":257}},"path":"/home/tskluzac/ext_repos/xtract-tika/.git/objects/b1/33ff0a43025f84998a061e9e15997b4b9b1d52","size":257},{"base_url":null,"crawl_timestamp":1635710555.671584,"file_id":"048d7fe3-e97a-47d4-83ad-bdd1b098d295","metadata":{"physical":{"extension":null,"path_type":"globus","size":385}},"path":"/home/tskluzac/ext_repos/xtract-tika/.git/objects/84/bd8631ec064552f91e89f074a137ac2bd399e8","size":385},{"base_url":null,"crawl_timestamp":1635710549.947026,"file_id":"f6

b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'
b'{"crawl_id":"0f5e6cd2-0402-4dff-b5ce-f18253b4c89c","file_ls":[],"num_files":0,"queue_empty":true}\n'


KeyboardInterrupt: 

In [ ]:
print(f"Tokens: {tokens}")

# HERE WE WILL TEST CONFIGURING OUR ENDPOINT. 
config_status = requests.post(f"{eb_url}/configure_ep/{funcx_ep_id}", json={'headers': fx_headers, 
                                                                            'timeout': 25, 
                                                                            'ep_name': 'tyler_test_ep_2', 
                                                                            'globus_eid': '12345', 
                                                                            'xtract_path':'/Users/tylerskluzacek/.xtract',
                                                                            'local_download_path': 'foobar',
                                                                            'local_mdata_path': '/Users/tylerskluzacek/Desktop/metadata'
                                                                     })
config_content = json.loads(config_status.content)
print(f"Returned: {config_content}")


## Step 3b: Xtract

Next we launch a non-blocking metadata extraction workflow that will automatically find all groups generated from our crawl_id, ship parsers to our endpoint as funcX, transfer the file (if necessary), and extract/send back metadata to the central Xtract service. This will just run constantly until the crawl is done and there are crawled groups left to extract. 

In [553]:
fx_headers = {'Authorization': f"Bearer {auths[funcx_scope].access_token}",
             'Search': auths['search'].authorizer.access_token,
             'Openid': auths['openid'].access_token}

xtract = requests.post(f'{eb_extract_url}/extract', json={
    'crawl_id': crawl_id, 
    'tokens': fx_headers, 
    'local_mdata_path': local_mdata_dir, 
    'remote_mdata_path': remote_mdata_dir})
print(f"Xtract response (should be 200): {xtract}")


Xtract response (should be 200): <Response [200]>


In [560]:
xtract_status = requests.get(f'{eb_extract_url}/get_extract_status', json={'crawl_id': crawl_id})
print(f"Xtract Status: {json.loads(xtract_status.content)['status']}")


Xtract Status: IN_SCHEDULING


## Step 4 (optional): Globus Search ingest

#### In this step we create (and name) a Globus Search index for our data.



In [ ]:
search_index = "ce2d9637-ad96-423f-99bc-935de889f640"
# COMING SOON -- via funcX

## Step 5: Metadata transfer (archive)

#### Metadata, by default, are stored on the filesystem of the machine on which they were extracted. Here we can move them to a Globus endpoint of our choosing. 

Here I will push the metadata to ALCF's Petrel data store and opt not to DELETE them from Jetstream. 

In [561]:
tokens = {"Transfer": auths["petrel"].access_token}

while True:
    xtract_status = requests.post(f'{eb_extract_url}/offload_mdata', json={
        'crawl_id': crawl_id, 
        'tokens': crawl_tokens, 
        'source_ep': source_ep_id, 
        'mdata_ep': mdata_ep_id, 
        'delete_source': False})

    response = json.loads(xtract_status.content)
    print(response['status'])
    if response['status'] == 'SUCCESS':
        break

SUCCESS


## Step 6: Let's query the index!

In [ ]:
# COMING SOON.